In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import shutil
from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
import warnings
from tensorflow.keras.backend import clear_session
warnings.filterwarnings('ignore')

In [ ]:
os.getcwd()

In [ ]:
train_dir = "Downloads/Emotion/train"
test_dir = "Downloads/Emotion/test"

SEED = 12
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64
EPOCHS = 30
FINE_TUNING_EPOCHS = 20
LR = 0.01
NUM_CLASSES = 7
EARLY_STOPPING_CRITERIA=3
CLASS_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]
CLASS_LABELS_EMOJIS = ["👿", "🤢" , "😱" , "😊" , "😐 ", "😔" , "😲" ]

In [97]:


train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rescale = 1./255,
                                   validation_split = 0.2
                                   
                                  
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2
                            
                                 )

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )

validation_generator = test_datagen.flow_from_directory(directory = train_dir,
                                                         target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                         batch_size = BATCH_SIZE,
                                                         shuffle  = True , 
                                                         color_mode = "grayscale",
                                                         class_mode = "categorical",
                                                         subset = "validation",
                                                         seed = 12
                                                        )

test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# Transfert learning

In [ ]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.DenseNet169(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3),
                                               include_top=False,
                                               weights="imagenet")(inputs)
    
    return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5) (x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)
    
    return x

def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)
    
    return classification_output

def define_compile_model():
    
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT ,IMG_WIDTH,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)
     
    model.compile(optimizer=tf.keras.optimizers.SGD(0.1), 
                loss='categorical_crossentropy',
                metrics = ['accuracy'])
  
    return model

In [ ]:
model1 = define_compile_model()
clear_output()

# Feezing the feature extraction layers
model1.layers[1].trainable = False

model1.summary()

In [93]:
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                         patience=EARLY_STOPPING_CRITERIA,
                                                         verbose= 1 ,
                                                         restore_best_weights=True
                                                        )
history1 = model1.fit(x = train_generator,
                    epochs = EPOCHS ,
                    validation_data = validation_generator , 
                    callbacks= [earlyStoppingCallback])

history1 = pd.DataFrame(history1.history)

359/359 [==============================] - 1984s 6s/step - loss: 1.2591 - accuracy: 0.5528 - val_loss: 1.2788 - val_accuracy: 0.5429
Epoch 7/30
359/359 [==============================] - 1031s 3s/step - loss: 1.2226 - accuracy: 0.5668 - val_loss: 1.2122 - val_accuracy: 0.5731
Epoch 8/30
359/359 [==============================] - 1081s 3s/step - loss: 1.2003 - accuracy: 0.5788 - val_loss: 1.1989 - val_accuracy: 0.5715
Epoch 9/30
359/359 [==============================] - 1092s 3s/step - loss: 1.1715 - accuracy: 0.5936 - val_loss: 1.2363 - val_accuracy: 0.5701
Epoch 10/30
359/359 [==============================] - 991s 3s/step - loss: 1.1420 - accuracy: 0.6068 - val_loss: 1.1850 - val_accuracy: 0.5846
Epoch 11/30
359/359 [==============================] - 1664s 5s/step - loss: 1.1254 - accuracy: 0.6141 - val_loss: 1.2945 - val_accuracy: 0.5452
Epoch 12/30
359/359 [==============================] - 1788s 5s/step - loss: 1.1042 - accuracy: 0.6208 - val_loss: 1.1669 - val_accuracy: 0.5898
E

In [94]:
# Un-Freezing the feature extraction layers for fine tuning 
model1.layers[1].trainable = True

model1.compile(optimizer=tf.keras.optimizers.SGD(0.001), #lower learning rate
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

history_ = model1.fit(x = train_generator,epochs = FINE_TUNING_EPOCHS ,validation_data = validation_generator)
history1 = history1.append(pd.DataFrame(history_.history) , ignore_index=True)

Epoch 1/20
359/359 [==============================] - 1121s 3s/step - loss: 1.0487 - accuracy: 0.6434 - val_loss: 1.0752 - val_accuracy: 0.6248
Epoch 2/20
359/359 [==============================] - 1110s 3s/step - loss: 1.0076 - accuracy: 0.6594 - val_loss: 1.0620 - val_accuracy: 0.6319
Epoch 3/20
359/359 [==============================] - 1085s 3s/step - loss: 0.9945 - accuracy: 0.6627 - val_loss: 1.0554 - val_accuracy: 0.6351
Epoch 4/20
359/359 [==============================] - 1123s 3s/step - loss: 0.9874 - accuracy: 0.6649 - val_loss: 1.0516 - val_accuracy: 0.6340
Epoch 5/20
359/359 [==============================] - 1156s 3s/step - loss: 0.9763 - accuracy: 0.6719 - val_loss: 1.0475 - val_accuracy: 0.6379
Epoch 6/20
359/359 [==============================] - 1143s 3s/step - loss: 0.9682 - accuracy: 0.6706 - val_loss: 1.0452 - val_accuracy: 0.6384
Epoch 7/20
359/359 [==============================] - 1110s 3s/step - loss: 0.9574 - accuracy: 0.6742 - val_loss: 1.0430 - val_accuracy:

In [95]:
model1.save("model_TL.h5")
model1.evaluate(test_generator)
preds1 = model1.predict(test_generator)
y_preds1 = np.argmax(preds1 , axis = 1 )
y_test = np.array(test_generator.labels)

113/113 [==============================] - 127s 1s/step


In [96]:
data1 = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_preds1
})
data1.to_csv('data_evaluation1.csv', index=False)
history1.to_csv('training1.csv', index=False)